In [1]:
import json
import os
import csv
import sys

csv.field_size_limit(sys.maxsize)

# hyperparams
M = 3
T = 2

data = []
with open(os.path.join("MNLI", "train.tsv")) as f:
    read_tsv = csv.reader(f, delimiter="\t")
    for row in read_tsv:
        data.append([row[-3], row[-1]])

data = data[1:]

patternMap = {}

print(data[0])
for ind, item in enumerate(data):
    if ind%500 == 0:
        print(ind, " of ", len(data), " done")
    hyp = item[0]
    label = item[1]
    hypList = hyp.split(" ")
    for index,token in enumerate(hypList):
        prefix = token.lower()
        for m in range(M,M+1):
            for t in range(1,T+1):
                #print("M: ", m, " T:", t)
                def recurse(prefix, t_left, m_left, word_ind):
                    #print(prefix)
                    if word_ind >= len(hypList)-1:
                        return
                    if m_left == 0:
                        if prefix not in patternMap:
                            patternMap[prefix] = {"count" : 0, "neutral" : 0, "entailment" : 0, "contradiction" : 0}
                        patternMap[prefix]["count"] += 1
                        patternMap[prefix][label] += 1
                        return
                    if t_left == 0:
                        recurse(prefix + " " + hypList[word_ind+1].lower(), T, m_left-1, word_ind+1)
                        return
                    else:
                        recurse(prefix + " " + hypList[word_ind+1].lower(), T, m_left-1, word_ind+1)
                        recurse(prefix + " _$_", t_left-1, m_left, word_ind+1)
                recurse(prefix, t, m-1, index)


#with open("pickle.json", "w") as f:
#    json.dump(patternMap, f)
                    
print(data[0:2])






['Product and geography are what make cream skimming work. ', 'neutral']
0  of  391176  done
500  of  391176  done
1000  of  391176  done
1500  of  391176  done
2000  of  391176  done
2500  of  391176  done
3000  of  391176  done
3500  of  391176  done
4000  of  391176  done
4500  of  391176  done
5000  of  391176  done
5500  of  391176  done
6000  of  391176  done
6500  of  391176  done
7000  of  391176  done
7500  of  391176  done
8000  of  391176  done
8500  of  391176  done
9000  of  391176  done
9500  of  391176  done
10000  of  391176  done
10500  of  391176  done
11000  of  391176  done
11500  of  391176  done
12000  of  391176  done
12500  of  391176  done
13000  of  391176  done
13500  of  391176  done
14000  of  391176  done
14500  of  391176  done
15000  of  391176  done
15500  of  391176  done
16000  of  391176  done
16500  of  391176  done
17000  of  391176  done
17500  of  391176  done
18000  of  391176  done
18500  of  391176  done
19000  of  391176  done
19500  of  3911

168500  of  391176  done
169000  of  391176  done
169500  of  391176  done
170000  of  391176  done
170500  of  391176  done
171000  of  391176  done
171500  of  391176  done
172000  of  391176  done
172500  of  391176  done
173000  of  391176  done
173500  of  391176  done
174000  of  391176  done
174500  of  391176  done
175000  of  391176  done
175500  of  391176  done
176000  of  391176  done
176500  of  391176  done
177000  of  391176  done
177500  of  391176  done
178000  of  391176  done
178500  of  391176  done
179000  of  391176  done
179500  of  391176  done
180000  of  391176  done
180500  of  391176  done
181000  of  391176  done
181500  of  391176  done
182000  of  391176  done
182500  of  391176  done
183000  of  391176  done
183500  of  391176  done
184000  of  391176  done
184500  of  391176  done
185000  of  391176  done
185500  of  391176  done
186000  of  391176  done
186500  of  391176  done
187000  of  391176  done
187500  of  391176  done
188000  of  391176  done


334000  of  391176  done
334500  of  391176  done
335000  of  391176  done
335500  of  391176  done
336000  of  391176  done
336500  of  391176  done
337000  of  391176  done
337500  of  391176  done
338000  of  391176  done
338500  of  391176  done
339000  of  391176  done
339500  of  391176  done
340000  of  391176  done
340500  of  391176  done
341000  of  391176  done
341500  of  391176  done
342000  of  391176  done
342500  of  391176  done
343000  of  391176  done
343500  of  391176  done
344000  of  391176  done
344500  of  391176  done
345000  of  391176  done
345500  of  391176  done
346000  of  391176  done
346500  of  391176  done
347000  of  391176  done
347500  of  391176  done
348000  of  391176  done
348500  of  391176  done
349000  of  391176  done
349500  of  391176  done
350000  of  391176  done
350500  of  391176  done
351000  of  391176  done
351500  of  391176  done
352000  of  391176  done
352500  of  391176  done
353000  of  391176  done
353500  of  391176  done


In [36]:
threshold = 1000

all_map = {}

classes = ["neutral", "entailment", "contradiction"]
for classType in classes:
    all_map[classType] = []

counter = 0
for k,v in patternMap.items():
    if counter%10000 == 0:
        print(counter, " of ", len(patternMap))
    counter += 1 
    for classType in classes:
        frac = v[classType]/v["count"]
        if v["count"] > 1000:
            all_map[classType].append([k, frac])

print("Done:")
for classType in classes:
    all_map[classType].sort(key= lambda x: x[1], reverse=True)


for classType in classes:
    print("Head for class: ", classType)
    all_map[classType] = all_map[classType][0:threshold]
    print(all_map[classType][0:3])

0  of  14071270
10000  of  14071270
20000  of  14071270
30000  of  14071270
40000  of  14071270
50000  of  14071270
60000  of  14071270
70000  of  14071270
80000  of  14071270
90000  of  14071270
100000  of  14071270
110000  of  14071270
120000  of  14071270
130000  of  14071270
140000  of  14071270
150000  of  14071270
160000  of  14071270
170000  of  14071270
180000  of  14071270
190000  of  14071270
200000  of  14071270
210000  of  14071270
220000  of  14071270
230000  of  14071270
240000  of  14071270
250000  of  14071270
260000  of  14071270
270000  of  14071270
280000  of  14071270
290000  of  14071270
300000  of  14071270
310000  of  14071270
320000  of  14071270
330000  of  14071270
340000  of  14071270
350000  of  14071270
360000  of  14071270
370000  of  14071270
380000  of  14071270
390000  of  14071270
400000  of  14071270
410000  of  14071270
420000  of  14071270
430000  of  14071270
440000  of  14071270
450000  of  14071270
460000  of  14071270
470000  of  14071270
480000

3840000  of  14071270
3850000  of  14071270
3860000  of  14071270
3870000  of  14071270
3880000  of  14071270
3890000  of  14071270
3900000  of  14071270
3910000  of  14071270
3920000  of  14071270
3930000  of  14071270
3940000  of  14071270
3950000  of  14071270
3960000  of  14071270
3970000  of  14071270
3980000  of  14071270
3990000  of  14071270
4000000  of  14071270
4010000  of  14071270
4020000  of  14071270
4030000  of  14071270
4040000  of  14071270
4050000  of  14071270
4060000  of  14071270
4070000  of  14071270
4080000  of  14071270
4090000  of  14071270
4100000  of  14071270
4110000  of  14071270
4120000  of  14071270
4130000  of  14071270
4140000  of  14071270
4150000  of  14071270
4160000  of  14071270
4170000  of  14071270
4180000  of  14071270
4190000  of  14071270
4200000  of  14071270
4210000  of  14071270
4220000  of  14071270
4230000  of  14071270
4240000  of  14071270
4250000  of  14071270
4260000  of  14071270
4270000  of  14071270
4280000  of  14071270
4290000  o

7670000  of  14071270
7680000  of  14071270
7690000  of  14071270
7700000  of  14071270
7710000  of  14071270
7720000  of  14071270
7730000  of  14071270
7740000  of  14071270
7750000  of  14071270
7760000  of  14071270
7770000  of  14071270
7780000  of  14071270
7790000  of  14071270
7800000  of  14071270
7810000  of  14071270
7820000  of  14071270
7830000  of  14071270
7840000  of  14071270
7850000  of  14071270
7860000  of  14071270
7870000  of  14071270
7880000  of  14071270
7890000  of  14071270
7900000  of  14071270
7910000  of  14071270
7920000  of  14071270
7930000  of  14071270
7940000  of  14071270
7950000  of  14071270
7960000  of  14071270
7970000  of  14071270
7980000  of  14071270
7990000  of  14071270
8000000  of  14071270
8010000  of  14071270
8020000  of  14071270
8030000  of  14071270
8040000  of  14071270
8050000  of  14071270
8060000  of  14071270
8070000  of  14071270
8080000  of  14071270
8090000  of  14071270
8100000  of  14071270
8110000  of  14071270
8120000  o

11370000  of  14071270
11380000  of  14071270
11390000  of  14071270
11400000  of  14071270
11410000  of  14071270
11420000  of  14071270
11430000  of  14071270
11440000  of  14071270
11450000  of  14071270
11460000  of  14071270
11470000  of  14071270
11480000  of  14071270
11490000  of  14071270
11500000  of  14071270
11510000  of  14071270
11520000  of  14071270
11530000  of  14071270
11540000  of  14071270
11550000  of  14071270
11560000  of  14071270
11570000  of  14071270
11580000  of  14071270
11590000  of  14071270
11600000  of  14071270
11610000  of  14071270
11620000  of  14071270
11630000  of  14071270
11640000  of  14071270
11650000  of  14071270
11660000  of  14071270
11670000  of  14071270
11680000  of  14071270
11690000  of  14071270
11700000  of  14071270
11710000  of  14071270
11720000  of  14071270
11730000  of  14071270
11740000  of  14071270
11750000  of  14071270
11760000  of  14071270
11770000  of  14071270
11780000  of  14071270
11790000  of  14071270
11800000  o

In [27]:
print(list(patternMap.keys())[0:10])

['product and geography', 'product and _$_ are', 'product and _$_ _$_ what', 'product _$_ geography are', 'product _$_ geography _$_ what', 'product _$_ geography _$_ _$_ make', 'product _$_ _$_ are what', 'product _$_ _$_ are _$_ make', 'product _$_ _$_ are _$_ _$_ cream', 'and geography are']


In [29]:
print(patternMap["there _$_ a _$_ _$_ the"])

{'count': 522, 'neutral': 108, 'entailment': 358, 'contradiction': 56}


In [32]:
import re
with open(os.path.join("MNLIOutliers", "outlierFileCLSEmbeddings.json")) as f:
    outlierData = json.load(f)
import copy

support = 0
none = 0
opposing = 0

class_wise = {}
for classType in classes:
    class_wise[classType] = {"total_c" : 0, "in_classified" : 0, "magnitude" : 0}
for item in outlierData:
    hypoth = item["hypothesis"]
    gt = item["gtLabel"]
    class_wise[gt]["total_c"] += 1
    counter = {}
    maxVal = 0
    maxClass = None
    for classType in classes:
        counter[classType] = 0
        regexes = [items[0] for items in all_map[classType]]
        
        for regex in regexes:
            processedRegex = ""
            regexList = regex.split(" ")
            for token in regexList:
                if token != "_$_":
                    token = token.replace("(", "")
                    token = token.replace(")", "")
                    if token == "(" or token == ")" or token == "[" or token == "]":
                        token = "" 
                    processedRegex = processedRegex + "\s*" + token + "\s*"
                else:
                    processedRegex = processedRegex + "\S*\s*"
            #print("Regex is: ", regex)
            #print("Processed regex is: " , processedRegex)
            counter[classType] += (len(re.findall(processedRegex,hypoth))*(patternMap[regex][classType]/patternMap[regex]["count"]))
        #print("For class:", classType, " val", counter[classType])
        if counter[classType] > maxVal:
            maxVal = counter[classType]
            maxClass = copy.deepcopy(classType )
    print(counter)
    print(maxVal,maxClass)
    for classType in classes:
        class_wise[classType]["magnitude"] += counter[classType]
    
    if maxClass == None:
        none += 1
        continue
    
    if maxClass == gt:
        class_wise[maxClass]["in_classified"] += 1
        support += 1
    else:
        opposing += 1

For class: neutral  val 29.436142424380932
For class: entailment  val 71.27590098886749
For class: contradiction  val 291.2879565867516
{'neutral': 29.436142424380932, 'entailment': 71.27590098886749, 'contradiction': 291.2879565867516}
291.2879565867516 contradiction
For class: neutral  val 15.281659811682118
For class: entailment  val 37.49546743954905
For class: contradiction  val 155.22287274876882
{'neutral': 15.281659811682118, 'entailment': 37.49546743954905, 'contradiction': 155.22287274876882}
155.22287274876882 contradiction
For class: neutral  val 25.98563748704231
For class: entailment  val 63.91361234882057
For class: contradiction  val 262.10075016413714
{'neutral': 25.98563748704231, 'entailment': 63.91361234882057, 'contradiction': 262.10075016413714}
262.10075016413714 contradiction
For class: neutral  val 25.985866164888566
For class: entailment  val 62.28981478558504
For class: contradiction  val 253.72431904952646
{'neutral': 25.985866164888566, 'entailment': 62.289

For class: entailment  val 58.82845091178281
For class: contradiction  val 242.41342165329957
{'neutral': 23.758127434917665, 'entailment': 58.82845091178281, 'contradiction': 242.41342165329957}
242.41342165329957 contradiction
For class: neutral  val 20.306757430910164
For class: entailment  val 50.04947641375866
For class: contradiction  val 206.64376615533118
{'neutral': 20.306757430910164, 'entailment': 50.04947641375866, 'contradiction': 206.64376615533118}
206.64376615533118 contradiction
For class: neutral  val 23.14780313043767
For class: entailment  val 56.46055022253299
For class: contradiction  val 231.39164664702938
{'neutral': 23.14780313043767, 'entailment': 56.46055022253299, 'contradiction': 231.39164664702938}
231.39164664702938 contradiction
For class: neutral  val 22.248275178557176
For class: entailment  val 54.97073507957642
For class: contradiction  val 226.7809897418664
{'neutral': 22.248275178557176, 'entailment': 54.97073507957642, 'contradiction': 226.7809897

For class: neutral  val 20.091064251430094
For class: entailment  val 49.49837415201489
For class: contradiction  val 204.41056159655497
{'neutral': 20.091064251430094, 'entailment': 49.49837415201489, 'contradiction': 204.41056159655497}
204.41056159655497 contradiction
For class: neutral  val 14.63458027324191
For class: entailment  val 35.842160654317745
For class: contradiction  val 148.52325907244034
{'neutral': 14.63458027324191, 'entailment': 35.842160654317745, 'contradiction': 148.52325907244034}
148.52325907244034 contradiction
For class: neutral  val 19.592024622652644
For class: entailment  val 48.51445779885108
For class: contradiction  val 199.89351757849624
{'neutral': 19.592024622652644, 'entailment': 48.51445779885108, 'contradiction': 199.89351757849624}
199.89351757849624 contradiction
For class: neutral  val 12.917475861099646
For class: entailment  val 31.75656661847656
For class: contradiction  val 131.3259575204238
{'neutral': 12.917475861099646, 'entailment': 31

For class: neutral  val 48.3305774882373
For class: entailment  val 118.27489556393371
For class: contradiction  val 480.394526947829
{'neutral': 48.3305774882373, 'entailment': 118.27489556393371, 'contradiction': 480.394526947829}
480.394526947829 contradiction
For class: neutral  val 15.71304617064226
For class: entailment  val 38.59767196303659
For class: contradiction  val 159.68928186632112
{'neutral': 15.71304617064226, 'entailment': 38.59767196303659, 'contradiction': 159.68928186632112}
159.68928186632112 contradiction
For class: neutral  val 22.63784195818735
For class: entailment  val 54.98847497862744
For class: contradiction  val 226.37368306318524
{'neutral': 22.63784195818735, 'entailment': 54.98847497862744, 'contradiction': 226.37368306318524}
226.37368306318524 contradiction
For class: neutral  val 25.483672870758216
For class: entailment  val 63.23726900573295
For class: contradiction  val 260.27905812350883
{'neutral': 25.483672870758216, 'entailment': 63.2372690057

For class: contradiction  val 239.2407028906574
{'neutral': 25.503432825417796, 'entailment': 60.25586428392484, 'contradiction': 239.2407028906574}
239.2407028906574 contradiction
For class: neutral  val 19.875650204276415
For class: entailment  val 48.90861020039497
For class: contradiction  val 202.21573959532864
{'neutral': 19.875650204276415, 'entailment': 48.90861020039497, 'contradiction': 202.21573959532864}
202.21573959532864 contradiction
For class: neutral  val 41.659770369073115
For class: entailment  val 102.56264637745049
For class: contradiction  val 421.7775832534763
{'neutral': 41.659770369073115, 'entailment': 102.56264637745049, 'contradiction': 421.7775832534763}
421.7775832534763 contradiction
For class: neutral  val 13.133169040579714
For class: entailment  val 32.30766888022033
For class: contradiction  val 133.55916207919998
{'neutral': 13.133169040579714, 'entailment': 32.30766888022033, 'contradiction': 133.55916207919998}
133.55916207919998 contradiction
For 

...

For class: entailment  val 147.7990403058639
For class: contradiction  val 606.5545250080102
{'neutral': 59.646434686125914, 'entailment': 147.7990403058639, 'contradiction': 606.5545250080102}
606.5545250080102 contradiction
For class: neutral  val 27.90522812817018
For class: entailment  val 68.07008257488116
For class: contradiction  val 280.0246892969487
{'neutral': 27.90522812817018, 'entailment': 68.07008257488116, 'contradiction': 280.0246892969487}
280.0246892969487 contradiction
For class: neutral  val 15.290100835380409
For class: entailment  val 37.81869149765801
For class: contradiction  val 155.8912076669616
{'neutral': 15.290100835380409, 'entailment': 37.81869149765801, 'contradiction': 155.8912076669616}
155.8912076669616 contradiction
For class: neutral  val 36.95540713740366
For class: entailment  val 91.71044965476719
For class: contradiction  val 377.3341432078291
{'neutral': 36.95540713740366, 'entailment': 91.71044965476719, 'contradiction': 377.3341432078291}
377

For class: entailment  val 210.39570677251936
For class: contradiction  val 860.3275001264356
{'neutral': 86.2767931010449, 'entailment': 210.39570677251936, 'contradiction': 860.3275001264356}
860.3275001264356 contradiction
For class: neutral  val 41.12150994966868
For class: entailment  val 102.0244427676989
For class: contradiction  val 419.8540472826324
{'neutral': 41.12150994966868, 'entailment': 102.0244427676989, 'contradiction': 419.8540472826324}
419.8540472826324 contradiction
For class: neutral  val 31.723057362393835
For class: entailment  val 78.92856247692276
For class: contradiction  val 324.34838016068346
{'neutral': 31.723057362393835, 'entailment': 78.92856247692276, 'contradiction': 324.34838016068346}
324.34838016068346 contradiction
For class: neutral  val 13.568054178170712
For class: entailment  val 33.412928279281196
For class: contradiction  val 138.0190175425481
{'neutral': 13.568054178170712, 'entailment': 33.412928279281196, 'contradiction': 138.01901754254

For class: contradiction  val 304.33200293554995
{'neutral': 29.85324992784777, 'entailment': 73.81474713660229, 'contradiction': 304.33200293554995}
304.33200293554995 contradiction
For class: neutral  val 29.98259329098285
For class: entailment  val 71.46384494644686
For class: contradiction  val 287.55356176257027
{'neutral': 29.98259329098285, 'entailment': 71.46384494644686, 'contradiction': 287.55356176257027}
287.55356176257027 contradiction
For class: neutral  val 33.690010513100354
For class: entailment  val 82.0527441814712
For class: contradiction  val 334.2572453054284
{'neutral': 33.690010513100354, 'entailment': 82.0527441814712, 'contradiction': 334.2572453054284}
334.2572453054284 contradiction
For class: neutral  val 23.542155123111204
For class: entailment  val 58.31601033991519
For class: contradiction  val 240.14183453697362
{'neutral': 23.542155123111204, 'entailment': 58.31601033991519, 'contradiction': 240.14183453697362}
240.14183453697362 contradiction
For clas

For class: contradiction  val 145.34897905751194
{'neutral': 14.855320623315402, 'entailment': 35.79570031917265, 'contradiction': 145.34897905751194}
145.34897905751194 contradiction
For class: neutral  val 20.091064251430094
For class: entailment  val 49.49837415201489
For class: contradiction  val 204.41056159655497
{'neutral': 20.091064251430094, 'entailment': 49.49837415201489, 'contradiction': 204.41056159655497}
204.41056159655497 contradiction
For class: neutral  val 21.368454880013235
For class: entailment  val 52.445641733135176
For class: contradiction  val 216.1859033868516
{'neutral': 21.368454880013235, 'entailment': 52.445641733135176, 'contradiction': 216.1859033868516}
216.1859033868516 contradiction
For class: neutral  val 15.290100835380409
For class: entailment  val 37.81869149765801
For class: contradiction  val 155.8912076669616
{'neutral': 15.290100835380409, 'entailment': 37.81869149765801, 'contradiction': 155.8912076669616}
155.8912076669616 contradiction
For 

For class: neutral  val 22.18772460610249
For class: entailment  val 53.382258997713905
For class: contradiction  val 218.43001639618362
{'neutral': 22.18772460610249, 'entailment': 53.382258997713905, 'contradiction': 218.43001639618362}
218.43001639618362 contradiction
For class: neutral  val 22.24799604623079
For class: entailment  val 55.00939676945257
For class: contradiction  val 226.74260718431665
{'neutral': 22.24799604623079, 'entailment': 55.00939676945257, 'contradiction': 226.74260718431665}
226.74260718431665 contradiction
For class: neutral  val 19.078585238065703
For class: entailment  val 46.65671541854268
For class: contradiction  val 191.26469934339158
{'neutral': 19.078585238065703, 'entailment': 46.65671541854268, 'contradiction': 191.26469934339158}
191.26469934339158 contradiction
For class: neutral  val 58.26952095816229
For class: entailment  val 139.89232686852657
For class: contradiction  val 563.838152173311
{'neutral': 58.26952095816229, 'entailment': 139.89

In [33]:
print(support, none, opposing)
print(class_wise)

722 0 1279
{'neutral': {'total_c': 629, 'in_classified': 0, 'magnitude': 48067.21649852182}, 'entailment': {'total_c': 650, 'in_classified': 0, 'magnitude': 117911.88380553256}, 'contradiction': {'total_c': 722, 'in_classified': 722, 'magnitude': 483923.8996959459}}


In [ ]:
test = "hello  this is LOL max"
import re
xx = "hello  this is LOL max"
r1 = re.findall("hello\s*\S*\sis",xx)
print(r1)